## 概要
 1. language modelsを使ってみる
 2. OutputParsersを使ってみる
 3. PromptTemplateを使ってみる
 4. LCELを使ってみる
 5. 会話履歴を使ってみる

### リンク
 - <a href="https://python.langchain.com/v0.2/docs/tutorials/llm_chain/" target=_blank>Langchain Tutorial</a>
 - <a href="https://pypi.org/project/langchain/#history" target=_black>Langchain Version</a>

In [15]:
import os
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)
from langchain_core.messages import (
    HumanMessage, 
    AIMessage,
    SystemMessage
)
from dotenv import load_dotenv
load_dotenv('../.env')

True

### 1. language modelsを使ってみる

#### 1-1. langchainを使わない場合

 - <a href="https://learn.microsoft.com/ja-jp/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python" target=_blank>モデルの実行</a>
 - <a href="https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release" target=_blank>Azure API Versionの確認</a>
 - <a href="https://platform.openai.com/docs/models" target=_blank>OpenAIのモデルリスト</a>

In [29]:
import os
from openai import (
    OpenAI,
    AzureOpenAI
)
# Azureの場合
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    client = AzureOpenAI(api_version="2024-12-01-preview")
    model_name = "gpt-4o" # Azureでデプロイしたモデル名
# OpenAIの場合
elif os.getenv('OPENAI_API_KEY') != "":
    client = OpenAI()
    model_name = "gpt-4" # OpenAIのモデル名 
else:
    print("APIKeyの設定を確認してください")

In [ ]:
response = client.chat.completions.create(
    model=model_name, 
    # ロールはsystem, user, assistantを指定する。contentはユーザーが入力したテキスト。
    messages=[
        {"role": "system", "content": "あなたは役に立つアシスタントだ。"},
        {"role": "user", "content": "Azure OpenAIはcustomer managed keysをサポートしていますか？"},
        {"role": "assistant", "content": "はい、customer managed keysはAzure OpenAIでサポートされています。"},
        {"role": "user", "content": "他のAzure AIサービスもこれをサポートしていますか？"}
    ]
)

print(response.choices[0].message.content)

はい、Azureの他の多くのAIサービスもCustomer Managed Keys (CMK) をサポートしています。CMKを使用すると、顧客がサービス内で利用される暗号化キーを管理、保持、回転することができるため、データのセキュリティやコンプライアンス要件をさらに強化することが可能です。

たとえば、以下のAzure AIサービスがCMKをサポートしています：
1. **Azure Cognitive Services**: 一部のCognitive Services（例えば、Azure Text AnalyticsやAzure Translatorなど）は、顧客が管理するキーを使用して機密データを保護する機能を備えています。
2. **Azure Machine Learning**: このサービスは、キーを使用してストレージアカウント、コンテナレジストリなどのリソースを暗号化するためのCMKをサポートしています。

### CMKの典型的な使用目的：
- データ暗号化におけるカスタマイズ性の向上。
- コンプライアンス基準や規制要件への対応（例：GDPRやHIPAA）。
- キーのライフサイクル管理（ローテーション、監査など）。

ただし、具体的なサービスがCMKをサポートしているかどうかと、その対応方法については、該当サービスのドキュメントを参照することをおすすめします。
はい、Azureの他の多くのAIサービスもCustomer Managed Keys (CMK) をサポートしています。CMKを使用すると、顧客がサービス内で利用される暗号化キーを管理、保持、回転することができるため、データのセキュリティやコンプライアンス要件をさらに強化することが可能です。

たとえば、以下のAzure AIサービスがCMKをサポートしています：
1. **Azure Cognitive Services**: 一部のCognitive Services（例えば、Azure Text AnalyticsやAzure Translatorなど）は、顧客が管理するキーを使用して機密データを保護する機能を備えています。
2. **Azure Machine Learning**: このサービスは、キーを使用してストレージアカウント、コンテナレジストリなどのリソースを暗号化する

#### 1-2. langchainを使う場合

<a href="https://python.langchain.com/v0.2/docs/tutorials/llm_chain/#using-language-models" target=_blank>モデルの実行</a>

In [33]:
# LLMの定義
model = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    model = AzureChatOpenAI(
        azure_deployment="gpt-4o",
        openai_api_version="2024-12-01-preview",
    )
else:
    print("APIKeyの設定を確認してください")

In [34]:
# Langchainでは、SystemMessage, HumanMessage, AIMessageのメッセージタイプを使用する。

messages = [
    SystemMessage(content="あなたは役に立つアシスタントだ。"),
    HumanMessage(content="Azure OpenAIはcustomer managed keysをサポートしていますか？"),
    AIMessage(content="はい、customer managed keysはAzure OpenAIでサポートされています。"),
    HumanMessage(content="他のAzure AIサービスもこれをサポートしていますか？")
]

result = model.invoke(messages)
print(result)

content='はい、いくつかのAzure AIサービスでは、**Customer Managed Keys (CMK)** をサポートしています。CMKを使用すると、Azure Key Vaultで管理されている独自の暗号化キーを用いてデータを暗号化し、データのセキュリティをさらに強化できます。以下は、CMKをサポートしている主なAzure AIサービスの例です：\n\n### 1. **Azure Cognitive Services**\n   - Azure Cognitive Servicesの一部のサービス（例えば、**Azure Form Recognizer** や **Azure Translator**）では、CMKを使用してデータ暗号化をカスタマイズできます。\n   - サポート状況はサービスごとに異なりますので、特定のサービスのドキュメントを確認することをお勧めします。\n\n### 2. **Azure Machine Learning**\n   - Azure Machine Learningでは、CMKを使用して機械学習ワークスペース内のデータを保護できます。\n   - モデル、実験データ、ストレージアカウントなどのリソースにCMKを適用できます。\n\n### 3. **Azure Synapse Analytics**\n   - Synapse Analyticsでは、CMKを使用して顧客データの暗号化をカスタマイズできます。これは、AIモデルを活用したデータ分析やインテリジェンス構築で重要です。\n\n### 4. **Azure Data Services**\n   - Azure AIサービスと連携することが多い **Azure Blob Storage** や **Azure SQL Database** などのデータサービスもCMKをサポートしており、AIソリューション全体のセキュリティを強化することができます。\n\n---\n\nCMKをサポートしているかどうかは、サービスの特定の機能やコンフィギュレーションに依存する場合がありますので、Azureの公式ドキュメントや最新情報を参照することをお勧めします。\n\n' response_metadata={'token_usage': {'completion

### 2. OutputParsersを使ってみる
AIの回答のフォーマットを指定して取得する

In [36]:
from langchain_core.output_parsers import StrOutputParser

In [37]:
parser = StrOutputParser()
parser.invoke(result)

'はい、いくつかのAzure AIサービスでは、**Customer Managed Keys (CMK)** をサポートしています。CMKを使用すると、Azure Key Vaultで管理されている独自の暗号化キーを用いてデータを暗号化し、データのセキュリティをさらに強化できます。以下は、CMKをサポートしている主なAzure AIサービスの例です：\n\n### 1. **Azure Cognitive Services**\n   - Azure Cognitive Servicesの一部のサービス（例えば、**Azure Form Recognizer** や **Azure Translator**）では、CMKを使用してデータ暗号化をカスタマイズできます。\n   - サポート状況はサービスごとに異なりますので、特定のサービスのドキュメントを確認することをお勧めします。\n\n### 2. **Azure Machine Learning**\n   - Azure Machine Learningでは、CMKを使用して機械学習ワークスペース内のデータを保護できます。\n   - モデル、実験データ、ストレージアカウントなどのリソースにCMKを適用できます。\n\n### 3. **Azure Synapse Analytics**\n   - Synapse Analyticsでは、CMKを使用して顧客データの暗号化をカスタマイズできます。これは、AIモデルを活用したデータ分析やインテリジェンス構築で重要です。\n\n### 4. **Azure Data Services**\n   - Azure AIサービスと連携することが多い **Azure Blob Storage** や **Azure SQL Database** などのデータサービスもCMKをサポートしており、AIソリューション全体のセキュリティを強化することができます。\n\n---\n\nCMKをサポートしているかどうかは、サービスの特定の機能やコンフィギュレーションに依存する場合がありますので、Azureの公式ドキュメントや最新情報を参照することをお勧めします。\n\n'

In [38]:
# contentからの取得
print(result.content)

はい、いくつかのAzure AIサービスでは、**Customer Managed Keys (CMK)** をサポートしています。CMKを使用すると、Azure Key Vaultで管理されている独自の暗号化キーを用いてデータを暗号化し、データのセキュリティをさらに強化できます。以下は、CMKをサポートしている主なAzure AIサービスの例です：

### 1. **Azure Cognitive Services**
   - Azure Cognitive Servicesの一部のサービス（例えば、**Azure Form Recognizer** や **Azure Translator**）では、CMKを使用してデータ暗号化をカスタマイズできます。
   - サポート状況はサービスごとに異なりますので、特定のサービスのドキュメントを確認することをお勧めします。

### 2. **Azure Machine Learning**
   - Azure Machine Learningでは、CMKを使用して機械学習ワークスペース内のデータを保護できます。
   - モデル、実験データ、ストレージアカウントなどのリソースにCMKを適用できます。

### 3. **Azure Synapse Analytics**
   - Synapse Analyticsでは、CMKを使用して顧客データの暗号化をカスタマイズできます。これは、AIモデルを活用したデータ分析やインテリジェンス構築で重要です。

### 4. **Azure Data Services**
   - Azure AIサービスと連携することが多い **Azure Blob Storage** や **Azure SQL Database** などのデータサービスもCMKをサポートしており、AIソリューション全体のセキュリティを強化することができます。

---

CMKをサポートしているかどうかは、サービスの特定の機能やコンフィギュレーションに依存する場合がありますので、Azureの公式ドキュメントや最新情報を参照することをお勧めします。




In [39]:
# resultの構造を表示
print(result.dict())

{'content': 'はい、いくつかのAzure AIサービスでは、**Customer Managed Keys (CMK)** をサポートしています。CMKを使用すると、Azure Key Vaultで管理されている独自の暗号化キーを用いてデータを暗号化し、データのセキュリティをさらに強化できます。以下は、CMKをサポートしている主なAzure AIサービスの例です：\n\n### 1. **Azure Cognitive Services**\n   - Azure Cognitive Servicesの一部のサービス（例えば、**Azure Form Recognizer** や **Azure Translator**）では、CMKを使用してデータ暗号化をカスタマイズできます。\n   - サポート状況はサービスごとに異なりますので、特定のサービスのドキュメントを確認することをお勧めします。\n\n### 2. **Azure Machine Learning**\n   - Azure Machine Learningでは、CMKを使用して機械学習ワークスペース内のデータを保護できます。\n   - モデル、実験データ、ストレージアカウントなどのリソースにCMKを適用できます。\n\n### 3. **Azure Synapse Analytics**\n   - Synapse Analyticsでは、CMKを使用して顧客データの暗号化をカスタマイズできます。これは、AIモデルを活用したデータ分析やインテリジェンス構築で重要です。\n\n### 4. **Azure Data Services**\n   - Azure AIサービスと連携することが多い **Azure Blob Storage** や **Azure SQL Database** などのデータサービスもCMKをサポートしており、AIソリューション全体のセキュリティを強化することができます。\n\n---\n\nCMKをサポートしているかどうかは、サービスの特定の機能やコンフィギュレーションに依存する場合がありますので、Azureの公式ドキュメントや最新情報を参照することをお勧めします。\n\n', 'additional_kwargs': {}, 'response_metada

### 3. PromptTemplateを使ってみる

#### 3-1. PromptTemplateを使わない場合

In [43]:
messages = [
    SystemMessage(content="イタリアの言語に翻訳してください"),
    HumanMessage(content="こんにちは"),
]
model.invoke(messages)

AIMessage(content='Ciao', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_material_text': {'detected': False, 'filtered': False}, 'self

In [45]:
messages = [
    SystemMessage(content="英語の言語に翻訳してください"),
    HumanMessage(content="こんにちは"),
]
model.invoke(messages)

AIMessage(content='Hello!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 21, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_material_text': {'detected': False, 'filtered': False}, 'se

#### 3-2. PromptTemplateを使う場合

In [51]:
from langchain_core.prompts import ChatPromptTemplate

In [58]:
# プロンプトの定義
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "{language}の言語に翻訳してください"),
        ("user", "{text}"),
    ]
)

In [59]:
# 辞書型で{}に入れる値を指定
prompt = prompt_template.invoke({"language": "イタリア", "text": "こんにちは"})
prompt

ChatPromptValue(messages=[SystemMessage(content='イタリアの言語に翻訳してください'), HumanMessage(content='こんにちは')])

In [60]:
# メッセージだけを抜粋
prompt.to_messages()

[SystemMessage(content='イタリアの言語に翻訳してください'), HumanMessage(content='こんにちは')]

In [61]:
# Promptを使ってみる
result = model.invoke(prompt)
print(result.content)

Ciao


In [63]:
# {luanguage}に英語、イタリア語、フランス語、中国語を入れて、順に実行し、実行結果のcontentをリストに格納して表示する
languages = ["イタリア", "フランス", "中国"]
results = []
for language in languages:
    prompt = prompt_template.invoke({"language": language, "text": "こんにちは"})
    result = model.invoke(prompt)
    results.append(result.content)

print(results)

['Ciao', 'Bonjour', 'こんにちは！中国語では「こんにちは」は以下のように翻訳されます：\n\n**你好 (Nǐ hǎo)**']


### 4. LCELを使ってみる
 今まで実行してきたように、Promtを設定 > モデルから回答取得 > 出力を成形と処理が続く  
 これをLCELと表記で、続けて記述することができます。  
 Chainという考え方になります。

#### 4-1. LCEL(Chain)を使わない場合

In [65]:
# PromptTemplateでPromptを作成
prompt = prompt_template.invoke({"language": "italian", "text": "こんにちは"})
# modelにpromptを渡して結果を取得
response = model.invoke(prompt)
# 結果を文字列に変換
result = parser.invoke(response)
print(result)

Ciao! 😊


#### 4-2.LCEL(Chain)を使った場合

In [67]:
# | を使って、PrompteTemplate, ChatOpenAI, StrOutputPerserをつなげる
chain = prompt_template | model | parser

In [68]:
# 後はChainを使って、結果を取得
chain.invoke({"language": "italian", "text": "こんにちは"})

'Ciao! 😊'

### 5. 会話履歴を使ってみる
チャットでは会話履歴が大事になってきます。  
会話履歴の取り扱いについて習得していきましょう

#### 5-1. 会話履歴の必要性
なぜ、会話の履歴が必要か確認していきます

In [69]:
# 一つ目のメッセージを入れてみます
model.invoke([HumanMessage(content="こんにちわ、もものきです")])

AIMessage(content='こんにちわ、もものきさん！😊どうぞよろしくお願いします。今日はどんなお話をしましょうか？質問や相談など、何でもお気軽にどうぞ！', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 15, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_m

In [71]:
# ２つめのメッセージをいれてみます
model.invoke([HumanMessage(content="私の名前を覚えていますか")])

AIMessage(content='申し訳ありませんが、私の記憶はセッションを超えて保持されません。そのため、あなたの名前を覚えていることはできません。ここで教えていただければ、会話の間だけその名前を使うことができます！', response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 15, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'fi

このように、1つずつメッセージを入れても、modelは適切な回答をしてくれません。  
過去の会話も続けて、modelに入れるとチャットのようなつづけた会話ができます

In [72]:
model.invoke(
    [
        HumanMessage(content="こんにちわ、もものきです"),
        AIMessage(content='こんにちわ、もものきさん、ご用件はありますか？'),
        HumanMessage(content="私の名前を覚えていますか？"),
    ]
)

AIMessage(content='もちろんです！あなたの名前は「もものき」さんですね。😊 何かお手伝いできることがあれば、どうぞお気軽にお話しください！', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 47, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_mater

#### 5-2. 会話履歴 その1(<a href="https://python.langchain.com/v0.2/docs/tutorials/chatbot/#message-history" target=_blank>v0.2 公式ドキュメント</a>)

In [73]:
!pip install langchain_community


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [77]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

# 会話履歴置き場
store = {}

# セッションIDを指定して履歴を取得
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 履歴を取得するためのRunnableWithMessageHistoryを作成
with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [78]:
# session_idのconfigを作成
# sessin_idは新しい会話を開始するたびに変更したりすると、会話ごとの履歴を保持することが出来る。
config = {"configurable": {"session_id": "session1"}}

In [80]:
# message_hisotryを使って、一つ目の質問を投げてみる
response = with_message_history.invoke(
    [HumanMessage(content="こんにちわ、もものきです")],
    config=config,
)
response.content

'こんにちわ、もものきさん！😊  \nお声がけありがとうございます。今日はどんなことについてお話ししたいですか？'

In [81]:
# message_historyをつかって２つ目の質問を投げてみる
response = with_message_history.invoke(
    [HumanMessage(content="私の名前を覚えていますか？")],
    config=config,
)
response.content

'もちろんです！😊 あなたの名前は「もものき」さんですね。お話しするたびにちゃんと覚えていますよ！ また何か気になることがあれば遠慮なく教えてくださいね。'

#### 5-3. 会話履歴その2 ConversationBufferMemoryなど
以前の方法

In [89]:
from langchain.memory import ConversationBufferMemory   
from langchain.chains import ConversationChain
chain = ConversationChain(
    llm=model,
    memory=ConversationBufferMemory()
    )

/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [90]:
chain.run([HumanMessage(content="こんにちわ、もものきです")])

/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


'こんにちは、もものきさん！お元気ですか？今日はどんなことをお話ししましょうか？日本語でおしゃべりするのは楽しいですね！気になっていることや好きなことについて教えていただければ嬉しいです。 😊'

In [91]:
chain.run([HumanMessage(content="私の名前を覚えていますか")])

'もちろん覚えていますよ！あなたの名前は「もものき」さんですね！素敵なお名前ですね。 😊 他に何かお話したいことや質問はありますか？'

#### 5-4. 会話履歴その3 スクラッチ
langchainは便利な機能がある一方、後ろで何をしているかわかりにくい場合もあります。  
また、更新が多いライブラリであり、方法が変わっていきます。  
次のコードでメッセージ履歴のイメージを把握しましょう！

In [92]:
# 会話履歴
messages = []

In [95]:
messages.append(HumanMessage(content="こんにちわ、もものきです"))
response = model.invoke(messages)
messages.append(AIMessage(content=response.content))
response

AIMessage(content='こんにちわ、もものきさん！😊  \nお元気ですか？今日はどんなことがお話ししたいですか？何かお手伝いできることがあれば教えてくださいね！', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 15, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protec

In [98]:
messages.append(HumanMessage(content="私の名前を覚えていますか？"))
response = model.invoke(messages)
messages.append(AIMessage(content=response.content))
response

AIMessage(content='もちろんです！もものきさんですよね 😊  \n以前お話ししたことも忘れませんよ！また何かお話ししたいことがあれば、ぜひ教えてくださいね。', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 91, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'prote

In [99]:
messages

[HumanMessage(content='こんにちわ、もものきです'),
 AIMessage(content='こんにちわ、もものきさん！😊  \nお元気ですか？今日はどんなことがお話ししたいですか？何かお手伝いできることがあれば教えてくださいね！'),
 HumanMessage(content='私の名前を覚えていますか？'),
 HumanMessage(content='私の名前を覚えていますか？'),
 AIMessage(content='もちろんです！もものきさんですよね 😊  \n以前お話ししたことも忘れませんよ！また何かお話ししたいことがあれば、ぜひ教えてくださいね。')]

#### 5-5. 会話履歴管理
  
会話を続けると、履歴が長くなり、modelに入れるトークン数を超えたり、トークン数によって費用が決まるため、費用がかさむ場合があります。  
  
そのため、会話履歴は管理する必要があります。  
  
5-3. 会話履歴その2で紹介したMemoryには、ConversationSummaryMemory, ConversationBufferWindowMemoryなどで会話履歴のトークン数を調整する方法があります。  
  

直近のドキュメントでは、trim_messagesなどがありますが、シンプルに配列で調整してもよい

In [103]:
messages = [
    SystemMessage(content="you are good assistant"),
    HumanMessage(content="hi! I am bob"),
    AIMessage(content="hi"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),

]
messages

[SystemMessage(content='you are good assistant'),
 HumanMessage(content='hi! I am bob'),
 AIMessage(content='hi'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2+2?'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem'),
 HumanMessage(content='having fun?')]

In [104]:
new_messages = [
    messages[0], # SystemMessage
    messages[-3], # HumanMessage
    messages[-2], # AIMessage
    messages[-1]  # HumanMessage
]
new_messages

[SystemMessage(content='you are good assistant'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem'),
 HumanMessage(content='having fun?')]

In [106]:
k = 3
new_message = [messages[0]]
new_message.extend(messages[-k:])
new_message

[SystemMessage(content='you are good assistant'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem'),
 HumanMessage(content='having fun?')]

In [107]:
response = model.invoke(new_message)
response

AIMessage(content="Always! I'm here to help and chat with you—whether it's answering questions, solving problems, or just having a friendly conversation. Are you having fun? 😊", response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 29, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severi